In [27]:
import pandas as pd
import os
import re

In [28]:
pwd = os.getcwd()
print(pwd)

e:\research\HEA_phase_classification\weighted_features\classification\new_alloy


In [29]:
# Load the data from the Excel file
df = pd.read_excel(pwd + "/weighted_features_new_alloys.xlsx")
df.head()

,Composition,EquiTibrium conditions,phase_count,PhaseType
0,Al0.25MoNbTiV,AC,s,BCC
1,Al0.5MoNbTiV,AC,S,BCC
2,Al0.75MoNbTiV,AC,S,BCC
3,Al1.5MoNbTiV,AC,S,BCC
4,AlMoNbTiV,AC,s,BCC


This script will output the unique elements in all the alloys and their maximum atomic fractions.

In [30]:
# Regular expression to capture element and atomic fraction
pattern = re.compile(r'([A-Z][a-z]*)(\d*\.?\d*)')

# Dictionary to hold max atomic fractions
max_atomic_fractions = {}

# Loop through the dataframe
for composition in df["Composition"]:
    matches = pattern.findall(composition)
    for match in matches:
        element = match[0]
        fraction = float(match[1]) if match[1] else 1  # if no number is found, assume 1
        if element not in max_atomic_fractions:
            max_atomic_fractions[element] = fraction
        else:
            max_atomic_fractions[element] = max(max_atomic_fractions[element], fraction)

print(max_atomic_fractions)

{'Al': 1.5, 'Mo': 2.0, 'Nb': 1.54, 'Ti': 2.0, 'V': 3.0, 'Ta': 1.54, 'Zr': 1.5, 'Hf': 1, 'Cr': 1.5, 'Co': 1, 'W': 1, 'Si': 0.5}


In [31]:
# Convert the max_atomic_fractions to DataFrame
df_max_atomic_fractions = pd.DataFrame.from_dict(max_atomic_fractions, orient='index', columns=['Max Atomic Fraction'])
df_max_atomic_fractions = df_max_atomic_fractions.sort_values(by="Max Atomic Fraction", ascending=False)
df_max_atomic_fractions

,Max Atomic Fraction
V,3.00
Mo,2.00
Ti,2.00
Nb,1.54
Ta,1.54
Al,1.50
Zr,1.50
Cr,1.50
Hf,1.00
Co,1.00


In [32]:
import itertools
import pandas as pd

# Define the base elements and other elements
base_elements = ['V', 'Mo', 'Ti', 'Nb', 'Ta', 'Al']
other_elements = ['Zr', 'Cr', 'W', 'Hf', 'Co']

def generate_distributions(remaining, num_elements):
    if num_elements == 1:
        return [[remaining]]
    distributions = []
    for i in range(5, remaining, 5):  # Adjusted increment to 5
        for rest in generate_distributions(remaining - i, num_elements - 1):
            distributions.append([i] + rest)
    return distributions

data = []

# Loop through the base elements
for base in base_elements:
    # Vary the base element's fraction
    for base_fraction in range(40, 76, 5):  # Adjusted increment to 5
        remaining_fraction = 100 - base_fraction
        # Choose four elements from the 'other_elements' list
        for comb in itertools.combinations(other_elements, 4):
            # Generate the possible distributions of the remaining_fraction among the 4 elements
            for distribution in generate_distributions(remaining_fraction, 4):
                HEA = {base: base_fraction}
                for el, frac in zip(comb, distribution):
                    HEA[el] = frac
                data.append(HEA)

# Create a dataframe from the data
df_HEA = pd.DataFrame(data).fillna(0)

# Convert the dataframe rows to HEA formulas
def row_to_formula(row):
    max_val = row.max()
    formula = ""
    for idx, val in row.iteritems():
        if val != 0:
            norm_val = round(val / max_val, 2)
            formula += f"{idx}{norm_val if norm_val != 1.0 else ''}"
    return formula

df_HEA['HEA_formula'] = df_HEA.apply(row_to_formula, axis=1)

# Select only the HEA_formula column
df_HEA = df_HEA[['HEA_formula']]

df_HEA.head()

C:\Users\Tohid\AppData\Local\Temp\ipykernel_17272\4134847961.py:40: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, val in row.iteritems():


,HEA_formula
0,V0.89Zr0.11Cr0.11W0.11Hf
1,VZr0.12Cr0.12W0.25Hf
2,VZr0.12Cr0.12W0.38Hf0.88
3,VZr0.12Cr0.12W0.5Hf0.75
4,VZr0.12Cr0.12W0.62Hf0.62


In [33]:
print(df_HEA.shape)

(14820, 1)


In [34]:
from pymatgen.core import Composition

In [36]:
# Define an empty list to store the elements of each composition
elements_list = []

# Loop over the compositions in the 'Composition' column
for composition in df_HEA['HEA_formula']:
    # Create a Pymatgen Composition object from the composition string
    pymatgen_comp = Composition(composition)
    # Extract the elements from the Composition object
    elements = [element.symbol for element in pymatgen_comp.elements]
    # Append the list of elements to the elements_list
    elements_list.append(elements)

# Add a new column to the DataFrame containing the elements without mole fractions
df_HEA['Elements'] = elements_list

# Print the updated DataFrame
df_HEA.head()

,HEA_formula,Elements
0,V0.89Zr0.11Cr0.11W0.11Hf,"[V, Zr, Cr, W, Hf]"
1,VZr0.12Cr0.12W0.25Hf,"[V, Zr, Cr, W, Hf]"
2,VZr0.12Cr0.12W0.38Hf0.88,"[V, Zr, Cr, W, Hf]"
3,VZr0.12Cr0.12W0.5Hf0.75,"[V, Zr, Cr, W, Hf]"
4,VZr0.12Cr0.12W0.62Hf0.62,"[V, Zr, Cr, W, Hf]"


In [37]:
df_HEA.to_excel("df_HEA.xlsx", index=False)